In [1]:
%matplotlib inline
import os
from tomato.audio.AudioAnalyzer import AudioAnalyzer
from tomato.symbolic.SymbTrAnalyzer import SymbTrAnalyzer
from tomato.joint.JointAnalyzer import JointAnalyzer
import pylab
pylab.rcParams['figure.figsize'] = (20, 8)


JointAnalyzer assumes the individual audio analysis and score analysis is applied earlier. 

In [5]:
# instantiate
audio_filepath = os.path.join('ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede',
                              'f970f1e0-0be9-4914-8302-709a0eac088e',
                              'f970f1e0-0be9-4914-8302-709a0eac088e.mp3')
audio_feature_filepath = os.path.join('ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede',
                                      'f970f1e0-0be9-4914-8302-709a0eac088e',
                                      'f970f1e0-0be9-4914-8302-709a0eac088e.pkl')
audio_features = AudioAnalyzer.from_pickle(audio_feature_filepath)

# score filepaths
score_filepath = os.path.join('ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede',
                              'ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt')
score_feature_filepath = os.path.join('ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede',
                                      'ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.pkl')
score_features = SymbTrAnalyzer.from_pickle(score_feature_filepath)

# instantiate analyzer objects
jointAnalyzer = JointAnalyzer(verbose=True)
audioAnalyzer = AudioAnalyzer(verbose=True)


You can use the single line call "analyze," which does all the available analysis simultaneously

In [ ]:
# joint analysis
joint_features = jointAnalyzer.analyze(score_filepath, score_features,
                                       audio_filepath, audio_features['pitch'])

... or call all the methods individually

In [ ]:
# joint analysis
# joint, score-informed tonic and tempo estimation
tonic, tempo = jointAnalyzer.extract_tonic_tempo(score_filepath, score_features,
                                                 audio_filepath, audio_features['pitch'])

# section linking and note-level alignment
sections, notes, section_candidates = jointAnalyzer.align_audio_score(
    score_filepath, score_features, audio_filepath, audio_features['pitch'], tonic, tempo)

# aligned pitch filter
pitch_filtered, notes_filtered = jointAnalyzer.filter_pitch(audio_features['pitch'], notes)

# aligned note model
note_models, pitch_distribution, aligned_tonic = jointAnalyzer.get_note_models(
    pitch_filtered, notes_filtered, tonic['symbol'])


Then you can recompute the audio features using the filtered pitch and tonic

In [7]:
# recompute the audio features
# get the melodic progression model
melodic_progression = audioAnalyzer.get_melodic_progression(pitch_filtered)

# pitch class distribution
pitch_class_distribution = pitch_distribution.to_pcd()

# transposition (ahenk) identification
ahenk = audioAnalyzer.identify_ahenk(aligned_tonic, aligned_tonic['symbol'])


- Obtaining the melodic progression model of ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/f970f1e0-0be9-4914-8302-709a0eac088e/f970f1e0-0be9-4914-8302-709a0eac088e.mp3


NameError: name 'pitch_distribution' is not defined

In [13]:
pitch_filtered['procedure']

KeyError: 'procedure'